In [3]:
import os
import numpy as np
import time

In [4]:
folder = "/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/"
molog = folder + "PROJ-COEFF-1_0.MOLog"

cp2k_input = folder + "cp2k.inp"
basis_set_file = "/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/BR"

Gaussian type orbitals

$$\varphi_i(\vec{r}) = R_i(r) \cdot Y_{l_i, m_i}(\theta, \phi),$$

where $Y_{l_i, m_i}(\theta, \phi)$ is the real-valued spherical harmonic and

$$ R_i(r) = r^{l_i} \sum_{j=1}^{N} c_{ij} \cdot \exp(-\alpha_j \cdot r^2). $$

NB: the $r^{l_i}$ will cancel out with the same factor reversed in $Y_{l_i, m_i}(\theta, \phi)$.



In [151]:
def spherical_harmonic_real(l, m, r_vec):
    (x, y, z) = r_vec
    c = 1/(2*np.sqrt(np.pi))
    
    r = np.sqrt(x**2 + y**2 + z**2)
    
    # s orbitals
    if (l, m) == (0, 0):
        return c
    
    # p orbitals
    elif (l, m) == (1, -1):
        return c*np.sqrt(3)*y/r
    elif (l, m) == (1, 0):
        return c*np.sqrt(3)*z/r
    elif (l, m) == (1, 1):
        return c*np.sqrt(3)*x/r
    
    # d orbitals
    elif (l, m) == (2, -2):
        return c*np.sqrt(15)*x*y/r**2
    elif (l, m) == (2, -1):
        return c*np.sqrt(15)*y*z/r**2
    elif (l, m) == (2, 0):
        return c*np.sqrt(5)/2*(2*z**2-x**2-y**2)/r**2
    elif (l, m) == (2, 1):
        return c*np.sqrt(15)*z*x/r**2
    elif (l, m) == (2, 2):
        return c*np.sqrt(15)/2*(x**2-y**2)/r**2
    
    print("No spherical harmonic found for l=%d, m=%d" % (l, m))
    return 0

def spherical_harmonic(l, m, r_vec):
    (x, y, z) = r_vec
    c = 1/(2*np.sqrt(np.pi))
    
    # s orbitals
    if (l, m) == (0, 0):
        return c
    
    # p orbitals
    elif (l, m) == (1, -1):
        return c*np.sqrt(3)*y
    elif (l, m) == (1, 0):
        return c*np.sqrt(3)*z
    elif (l, m) == (1, 1):
        return c*np.sqrt(3)*x
    
    # d orbitals
    elif (l, m) == (2, -2):
        return c*np.sqrt(15)*x*y
    elif (l, m) == (2, -1):
        return c*np.sqrt(15)*y*z
    elif (l, m) == (2, 0):
        return c*np.sqrt(5)/2*(2*z**2-x**2-y**2)
    elif (l, m) == (2, 1):
        return c*np.sqrt(15)*z*x
    elif (l, m) == (2, 2):
        return c*np.sqrt(15)/2*(x**2-y**2)
    
    print("No spherical harmonic found for l=%d, m=%d" % (l, m))
    return 0
    

In [19]:
r_vec = np.array([1, 2, 3])

spherical_harmonic(1, -1, r_vec)

0.26116902826540894

In [206]:
def read_basis_functions(basis_set_file, cp2k_input_file):
    # Find the used basis sets for all used elements
    elem_basis = {}
    with open(cp2k_input_file) as f:
        lines = f.readlines()
        for i in range(len(lines)):
            parts = lines[i].split()
            if parts[0] == "&KIND":
                elem = parts[1]
                for j in range(10):
                    parts = lines[i+j].split()
                    if parts[0] == "BASIS_SET":
                        basis = parts[1]
                        elem_basis[elem] = basis
                        break
    
    basis_sets = {}
    with open(basis_set_file) as f:
        lines = f.readlines()
        for i in range(len(lines)):
            parts = lines[i].split()
            if parts[0] in elem_basis:
                elem = parts[0]
                if parts[1] == elem_basis[elem] or parts[2] == elem_basis[elem]:
                    # We have found the correct basis set
                    #print(lines[i], end='')
                    #print(lines[i+1], end='')
                    basis_functions = []
                    nsets = int(lines[i+1])
                    cursor = 2
                    for j in range(nsets):
                        comp = [int(x) for x in lines[i+cursor].split()]
                        #print(comp)
                        n_princ, l_min, l_max, n_exp = comp[:4]
                        l_arr = np.arange(l_min, l_max+1, 1)
                        n_basisf_for_l = comp[4:]
                        assert len(l_arr) == len(n_basisf_for_l)
                        
                        exps = []
                        coeffs = []
                        
                        for k in range(n_exp):
                            exp_c = [float(x) for x in lines[i+cursor+k+1].split()]
                            exps.append(exp_c[0])
                            coeffs.append(exp_c[1:])
                        
                        exps = np.array(exps)
                        coeffs = np.array(coeffs)
                        
                        indx = 0
                        for l, nl in zip(l_arr, n_basisf_for_l):
                            for i in range(nl):
                                #print("l =", l)
                                #print(exps)
                                #print(coeffs[:, indx])
                                basis_functions.append([l, exps, coeffs[:, indx]])
                                indx += 1
                        cursor += n_exp + 1
                        
                    #print()
                    basis_sets[elem] = basis_functions
                    
    return basis_sets         

In [208]:
basis_sets = read_basis_functions(basis_set_file, cp2k_input)

basis_sets

{'C': [[0,
   array([ 5.60533075,  2.11301639,  0.76991145,  0.34815709,  0.12821225]),
   array([ 0.11153288,  0.15314219, -0.32139688, -0.61092642, -0.13399086])],
  [0,
   array([ 5.60533075,  2.11301639,  0.76991145,  0.34815709,  0.12821225]),
   array([ 0.46650748,  0.14411321,  0.02936488,  0.92947039,  1.06691433])],
  [1,
   array([ 5.60533075,  2.11301639,  0.76991145,  0.34815709,  0.12821225]),
   array([-0.06486898, -0.20047026, -0.41314173, -0.41533714, -0.28852767])],
  [1,
   array([ 5.60533075,  2.11301639,  0.76991145,  0.34815709,  0.12821225]),
   array([-0.05547569, -0.21616518, -0.43662427,  0.7840007 ,  0.98790813])],
  [2,
   array([ 5.60533075,  2.11301639,  0.76991145,  0.34815709,  0.12821225]),
   array([ 0.02995277,  0.07078446,  0.67460106,  0.17365206,  1.03631702])]],
 'H': [[0,
   array([ 10.06846823,   2.68022287,   0.79150154,   0.23911615,   0.08219318]),
   array([-0.03391744, -0.12220221, -0.44381886, -0.45318219, -0.13161286])],
  [0,
   array([ 1

In [209]:
# Inspiration from
# https://github.com/ondrejkrejci/PPSTM/blob/master/pyPPSTM/ReadSTM.py#L520
def read_cp2k_MO_file(fn):
    print("Reading CP2K MOs from:"+fn)

    # read all lines into memory
    f = open(fn)
    lines = []
    for l in f.readlines():
        l = l.strip()
        if(len(l)==0): continue
        lines.append(l)
    f.close()
    
    # Remove all but the last section
    delete_line = False
    for i, line in reversed(list(enumerate(lines))):
        if delete_line:
            del lines[i]
        if "MO EIGENVALUES" in line:
            delete_line = True
    
    # detect dimensions
    parts = lines[-3].split()
    nbasis = int(parts[0])
    natoms = int(parts[1])
    nmos = int(lines[-nbasis-5].split()[-1])
    
    
    nlines_per_block = nbasis + 3
    nblocks = int((len(lines)-3)/nlines_per_block)
    
    
    print(nbasis, natoms, nmos, nblocks)
    
    print("Found %d MOs spanned by %d basis functions centered on %d atoms."%(nmos, nbasis, natoms))
    
    assert lines[-1].startswith("HOMO-LUMO gap:")
    assert lines[-2].startswith("Fermi energy:")
    fermi_energy = 27.211385 * float(lines[-2].split()[2])

    # unfold table
    idx = []
    evals = []
    occs = []
    evecs = [list() for i in range(nbasis)]
    labels = [l.split()[:4] for l in lines[4:nbasis+4]]
    
    first_line = 1
    
    for iblock in range(nblocks):
        a = first_line + iblock*nlines_per_block
        evals.extend(lines[a+1].split())
        occs.extend(lines[a+2].split())
        for j in range(nbasis):
            parts = lines[a+3+j].split()
            assert parts[:4] == labels[j]
            evecs[j].extend(parts[4:])

    # convert to numpy arrays
    evals = np.array(evals, float)
    occs = np.array(occs, float)
    evecs = np.array(evecs, float)
    assert evals.shape == (nmos,)
    assert occs.shape == (nmos,)
    assert evecs.shape == (nbasis, nmos)

    # convert hartree to eV
    evals = 27.211385 * evals
    # NB: evecs[basis_idx, morbital_idx] = overlap/projection/scalar_product
    
    #### --------------------------------------------------------------------
    #### Further processing into format
    #### molog_data[atom_nr] = ['H', ['3s', '3px', ...], [[evec for 3s], [evec for 3px], ...]
    molog_data = [[] for i in range(natoms)]
    
    for label, evec in zip(labels, evecs):
        atom_nr = int(label[1]) - 1
        elem = label[2]
        if len(molog_data[atom_nr]) == 0:
            molog_data[atom_nr].extend([elem, [], []])
        molog_data[atom_nr][1].append(label[3])
        molog_data[atom_nr][2].append(evec)
    
    return molog_data, evals, occs, fermi_energy



In [210]:
molog_output = read_cp2k_MO_file(molog)

molog_data, evals, occs, fermi_energy = molog_output
molog_data[1]

Reading CP2K MOs from:/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog
38 4 10 5
Found 10 MOs spanned by 38 basis functions centered on 4 atoms.


['C',
 ['2s',
  '3s',
  '3px',
  '3py',
  '3pz',
  '4px',
  '4py',
  '4pz',
  '4dx2',
  '4dxy',
  '4dxz',
  '4dy2',
  '4dyz',
  '4dz2'],
 [array([  4.71652007e-01,   3.17066375e-01,  -8.81880766e-04,
           1.08906900e-09,   1.50851510e-09,  -2.85404700e-09,
          -2.12357270e-09,   9.25870575e-02,   4.66577646e-01,
          -5.25645321e-01]),
  array([  9.72351226e-03,   4.20227216e-03,   1.29653777e-02,
          -1.23593100e-10,  -4.27809900e-10,   4.98023400e-10,
           3.91486500e-10,  -5.58998497e-02,  -7.90217826e-02,
           2.79169896e-01]),
  array([ -5.95457700e-10,  -1.57622880e-09,   3.64821300e-10,
           9.74077242e-02,  -5.76378637e-01,   8.21069724e-02,
          -6.84938097e-01,  -4.46075970e-09,  -1.15160650e-09,
           5.57577000e-11]),
  array([  3.03113300e-10,  -7.41652100e-10,   3.50779700e-10,
          -5.76378636e-01,  -9.74077236e-02,  -6.84938098e-01,
          -8.21069729e-02,  -4.02927500e-09,  -3.05029160e-09,
           3.1193580

In [211]:
# Assuming alphabetical order: x, y, z
# returns in order of increasing m
def cart_coef_to_spherical(l, coefs):
    if l == 0:
        assert len(coefs) == 1
        return np.array(coefs)
    elif l == 1:
        assert len(coefs) == 3
        return np.array([coefs[1], coefs[2], coefs[0]])
    elif l == 2:
        assert len(coefs) == 6
        conv_mat = np.array([[ 0.0, 1.0, 0.0,  0.0, 0.0, 0.0],
                             [ 0.0, 0.0, 0.0,  0.0, 1.0, 0.0],
                             [-0.5, 0.0, 0.0, -0.5, 0.0, 1.0],
                             [ 0.0, 0.0, 1.0,  0.0, 0.0, 0.0],
                             [ 0.5*np.sqrt(3), 0, 0, -0.5*np.sqrt(3), 0, 0]])
        return np.dot(conv_mat, coefs)
    else:
        print("Not implemented.")
        return 0.0
    

In [223]:

# morb_decomposition[morb_nr, atom_nr] = [coefs_for_2s, coefs_for_3s, coefs_for_3p, coefs_for_3d, ...]
# coefs_for_3p = [coef_for_m=-1, coef_for_m=0, coef_for_m=1]
def process_molog_output(molog_output):
    
    molog_data, evals, occs, fermi_energy = molog_output
    
    natoms = len(molog_data)
    nmos = len(evals)
    
    morb_decomposition = [[[] for j in range(natoms)] for i in range(nmos)]

    for i_at in range(natoms):
        elem = molog_data[i_at][0]
        orb_labels = molog_data[i_at][1]
        eig_vecs = molog_data[i_at][2]
        
        i_orb = 0
        while i_orb < len(orb_labels):
            
            n_orb = int(orb_labels[i_orb][0])
            cart_orb = orb_labels[i_orb][1:]
            
            if cart_orb == 's':
                eig_vec = eig_vecs[i_orb]
                for i_mo in range(nmos):
                    morb_decomposition[i_mo][i_at].append(cart_coef_to_spherical(0, [eig_vec[i_mo]]))
                i_orb += 1
                continue
            
            elif cart_orb == 'px':
                eig_px = eig_vecs[i_orb]
                eig_py = eig_vecs[i_orb+1]
                eig_pz = eig_vecs[i_orb+2]
                for i_mo in range(nmos):
                    spherical_coefs = cart_coef_to_spherical(1, [eig_px[i_mo], eig_py[i_mo], eig_pz[i_mo]])
                    morb_decomposition[i_mo][i_at].append(spherical_coefs)
                i_orb += 3
                continue
            
            elif cart_orb == 'dx2':
                eig_dx2 = eig_vecs[i_orb]
                eig_dxy = eig_vecs[i_orb+1]
                eig_dxz = eig_vecs[i_orb+2]
                eig_dy2 = eig_vecs[i_orb+3]
                eig_dyz = eig_vecs[i_orb+4]
                eig_dz2 = eig_vecs[i_orb+5]
                for i_mo in range(nmos):
                    spherical_coefs = cart_coef_to_spherical(2, [eig_dx2[i_mo], eig_dxy[i_mo], eig_dxz[i_mo],
                                                                 eig_dy2[i_mo], eig_dyz[i_mo], eig_dz2[i_mo]])
                    morb_decomposition[i_mo][i_at].append(spherical_coefs)
                i_orb += 6
                continue
            
            else:
                print('Error: found unsupported orbital label')
                break
                
    return morb_decomposition
        
morb_decomposition = process_molog_output(molog_output)

morb_decomposition[4][3]

[[array([ 0.1376726]),
  array([-0.00277589]),
  array([ -6.87475000e-11,  -2.85527972e-02,   4.46790000e-11])],
 [array([ 0.47165201]),
  array([ 0.00972351]),
  array([  3.03113300e-10,   1.58827292e-01,  -5.95457700e-10]),
  array([  2.17324100e-10,   3.31769755e-02,   7.26688000e-11]),
  array([  1.14750200e-10,  -1.87114300e-10,  -1.54450102e-02,
          -5.44750000e-11,  -4.23048220e-11])],
 [array([ 0.471652]),
  array([ 0.00972351]),
  array([  2.51000570e-09,  -1.58827299e-01,   2.10275770e-09]),
  array([  3.39177000e-11,  -3.31769760e-02,   1.58458000e-11]),
  array([ -9.85331000e-11,   1.99150300e-10,  -1.54450094e-02,
           1.01432400e-10,   8.71173056e-11])],
 [array([ 0.13767258]),
  array([-0.00277589]),
  array([ -1.80271000e-10,   2.85527952e-02,  -1.56630600e-10])]]

In [213]:
folder = "/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/"

file_xyz = folder + "c2h2/p.xyz"

file_molog = folder + "c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog"

file_basis_set = folder + "BR"

file_cp2k_inp = folder + "c2h2/morb_diag_cart/cp2k.inp"


ang_2_bohr = 1.0/0.52917721067

cell = np.array([[4.0, 0.0, 0.0], [0.0, 4.0, 0.0], [0.0, 0.0, 7.345]])
cell *= ang_2_bohr

step = 0.08
step *= ang_2_bohr

cell_n = np.array((int(cell[0, 0]/step), int(cell[1, 1]/step), int(cell[2, 2]/step)))

def grid_morbital(morbital_nr, file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n):
    
    # Read atomic positions (in angstrom)
    positions = np.genfromtxt(file_xyz, dtype=None, skip_header=2)
    
    # Read MOlog file for molecular orbital decompositon in atomic orbitals
    molog_output = read_cp2k_MO_file(file_molog)
    
    # Process the molog data 
    morb_decomposition = process_molog_output(molog_output)
    
    # Read basis set info
    basis_sets = read_basis_functions(file_basis_set, file_cp2k_inp)
    
    dv = cell/cell_n
    
    data = np.zeros(cell_n) 
    
    for xi in range(cell_n[0]):
        for yi in range(cell_n[1]):
            for zi in range(cell_n[2]):
                # In bohr
                (x, y, z) = xi*dv[0] + yi*dv[1] + zi*dv[2]
                
                for i_at in range(len(positions)):
                    elem = positions[i_at][0].decode("utf-8")
                    x_at = positions[i_at][1] * ang_2_bohr
                    y_at = positions[i_at][2] * ang_2_bohr
                    z_at = positions[i_at][3] * ang_2_bohr
                    
                    for bs, orb_coefs in zip(basis_sets[elem], morb_decomposition[morbital_nr][i_at]):
                        l = bs[0]
                        
                        es = bs[1]
                        cs = bs[2]
                        
                        r_vec = np.array([x-x_at, y-y_at, z-z_at])
                        r_vec_2 = np.dot(r_vec, r_vec)
                        
                        # Calculate the radial part
                        radial_part = 0
                        for e, c in zip(es, cs):
                            radial_part += c*np.exp(-1.0*e*r_vec_2)
                        
                        
                        for i, m in enumerate(range(-l, l+1, 1)):
                            coef = orb_coefs[i]
                            data[xi, yi, zi] += radial_part*coef*spherical_harmonic(l, m, r_vec)
    
    return data


def grid_atomic_orbitals(file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n):
    
    # Read atomic positions
    positions = np.genfromtxt(file_xyz, dtype=None, skip_header=2)
    
    # Read MOlog file for molecular orbital decompositon in atomic orbitals
    molog_output = read_cp2k_MO_file(file_molog)
    
    # Process the molog data 
    morb_decomposition = process_molog_output(molog_output)
    
    # Read basis set info
    basis_sets = read_basis_functions(file_basis_set, file_cp2k_inp)
    
    dv = cell/cell_n
    
    atomic_orbitals_on_grid = []
    
    for i_at in [0]:
        elem = positions[i_at][0].decode("utf-8")
        x_at = positions[i_at][1] * ang_2_bohr
        y_at = positions[i_at][2] * ang_2_bohr
        z_at = positions[i_at][3] * ang_2_bohr
        
        atomic_orbitals_on_grid.append([elem, np.array([x_at, y_at, z_at]), []])
        
        
        for bs in basis_sets[elem]:
            l = bs[0]
                        
            es = bs[1]
            cs = bs[2]
            
            atomic_orbitals_on_grid[-1][2].append([l, []])
            
            for i, m in enumerate(range(-l, l+1, 1)):
                data = np.zeros(cell_n) 
                
                for xi in range(cell_n[0]):
                    for yi in range(cell_n[1]):
                        for zi in range(cell_n[2]):
                            (x, y, z) = xi*dv[0] + yi*dv[1] + zi*dv[2]

                            r_vec = np.array([x-x_at, y-y_at, z-z_at])
                            r_vec_2 = np.dot(r_vec, r_vec)
                            
                            # Calculate the radial part
                            radial_part = 0
                            for e, c in zip(es, cs):
                                radial_part += c*np.exp(-1.0*e*r_vec_2)
                            
                            data[xi, yi, zi] += radial_part*spherical_harmonic(l, m, r_vec)
                            
                atomic_orbitals_on_grid[-1][2][-1][1].append(data)
                
                #fname = "./at%d_l%d_m%d.cube" % (i_at, l, m)
                #write_cube_file(fname, positions, cell, cell_n, data)
    
    return atomic_orbitals_on_grid

In [200]:
start_time = time.time()
data0 = grid_morbital(0, file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n)
print("time: ", (time.time()-start_time))
print("max: ", np.max(data))
print("min: ", np.min(data))


Reading CP2K MOs from:/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog
38 4 10 5
Found 10 MOs spanned by 38 basis functions centered on 4 atoms.
 H  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q1
 1
[2, 0, 1, 5, 2, 1]
l = 0
[ 10.06846823   2.68022287   0.79150154   0.23911615   0.08219318]
[-0.03391744 -0.12220221 -0.44381886 -0.45318219 -0.13161286]
l = 0
[ 10.06846823   2.68022287   0.79150154   0.23911615   0.08219318]
[ 0.05919378  0.84331833 -1.15570712  0.04947962  0.52270874]
l = 1
[ 10.06846823   2.68022287   0.79150154   0.23911615   0.08219318]
[ 0.00990513  0.12244957  0.47718324  0.54791968  0.86903185]

 C  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q4
 1
[2, 0, 2, 5, 2, 2, 1]
l = 0
[ 5.60533075  2.11301639  0.76991145  0.34815709  0.12821225]
[ 0.11153288  0.15314219 -0.32139688 -0.61092642 -0.13399086]
l = 0
[ 5.60533075  2.11301639  0.76991145  0.34815709  0.12821225]
[ 0.46650748  0.14411321  0.02936488  0.92947039  1.06691433]
l = 1
[

In [201]:
start_time = time.time()
data1 = grid_morbital(1, file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n)
print("time: ", (time.time()-start_time))

Reading CP2K MOs from:/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog
38 4 10 5
Found 10 MOs spanned by 38 basis functions centered on 4 atoms.
 H  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q1
 1
[2, 0, 1, 5, 2, 1]
l = 0
[ 10.06846823   2.68022287   0.79150154   0.23911615   0.08219318]
[-0.03391744 -0.12220221 -0.44381886 -0.45318219 -0.13161286]
l = 0
[ 10.06846823   2.68022287   0.79150154   0.23911615   0.08219318]
[ 0.05919378  0.84331833 -1.15570712  0.04947962  0.52270874]
l = 1
[ 10.06846823   2.68022287   0.79150154   0.23911615   0.08219318]
[ 0.00990513  0.12244957  0.47718324  0.54791968  0.86903185]

 C  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q4
 1
[2, 0, 2, 5, 2, 2, 1]
l = 0
[ 5.60533075  2.11301639  0.76991145  0.34815709  0.12821225]
[ 0.11153288  0.15314219 -0.32139688 -0.61092642 -0.13399086]
l = 0
[ 5.60533075  2.11301639  0.76991145  0.34815709  0.12821225]
[ 0.46650748  0.14411321  0.02936488  0.92947039  1.06691433]
l = 1
[

In [214]:
start_time = time.time()
data5 = grid_morbital(5, file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n)
print("time: ", (time.time()-start_time))

Reading CP2K MOs from:/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog
38 4 10 5
Found 10 MOs spanned by 38 basis functions centered on 4 atoms.
time:  92.15966129302979


In [215]:
start_time = time.time()
data6 = grid_morbital(6, file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n)
print("time: ", (time.time()-start_time))

Reading CP2K MOs from:/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog
38 4 10 5
Found 10 MOs spanned by 38 basis functions centered on 4 atoms.
time:  94.60755705833435


In [221]:
# Calculate overlap between two orbitals
# vol_elem in bohr!
def overlap(data1, data2, vol_elem):
    return (np.dot(data1.flatten(), data2.flatten())*vol_elem)**2


vol_elem = np.linalg.det(cell/cell_n)
overlap(data5, data6, vol_elem)

6.679440779876176e-08

In [170]:
start_time = time.time()
atomic_orbitals_on_grid = grid_atomic_orbitals(file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n)
print("time: ", (time.time()-start_time))
#print("max: ", np.max(data))
#print("min: ", np.min(data))

positions = np.genfromtxt(file_xyz, dtype=None, skip_header=2)

Reading CP2K MOs from:/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog
38 4 10 5
Found 10 MOs spanned by 38 basis functions centered on 4 atoms.
 H  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q1
 1
[2, 0, 1, 5, 2, 1]
l = 0
[ 10.06846823   2.68022287   0.79150154   0.23911615   0.08219318]
[-0.03391744 -0.12220221 -0.44381886 -0.45318219 -0.13161286]
l = 0
[ 10.06846823   2.68022287   0.79150154   0.23911615   0.08219318]
[ 0.05919378  0.84331833 -1.15570712  0.04947962  0.52270874]
l = 1
[ 10.06846823   2.68022287   0.79150154   0.23911615   0.08219318]
[ 0.00990513  0.12244957  0.47718324  0.54791968  0.86903185]

 C  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q4
 1
[2, 0, 2, 5, 2, 2, 1]
l = 0
[ 5.60533075  2.11301639  0.76991145  0.34815709  0.12821225]
[ 0.11153288  0.15314219 -0.32139688 -0.61092642 -0.13399086]
l = 0
[ 5.60533075  2.11301639  0.76991145  0.34815709  0.12821225]
[ 0.46650748  0.14411321  0.02936488  0.92947039  1.06691433]
l = 1
[

In [197]:
# [atom][2][l=0, 0, 1, ...][1][m]
data1 = atomic_orbitals_on_grid[0][2][2][1][0]
data2 = atomic_orbitals_on_grid[0][2][2][1][2]
data3 = atomic_orbitals_on_grid[0][2][1][1][0]
data4 = atomic_orbitals_on_grid[0][2][0][1][0]

# Calculate overlap between two orbitals
# vol_elem in bohr!
def overlap(data1, data2, vol_elem):
    return (np.dot(data1.flatten(), data2.flatten())*vol_elem)**2


vol_elem = np.linalg.det(cell/cell_n)
overlap(data0, data1, vol_elem)


0.0042338127552316134

In [51]:
filename = './output.cube'

def write_cube_file(filename, atoms, cell, cell_n, data):
    
    natoms = len(atoms)
    origin = np.array([0.0, 0.0, 0.0])
    origin *= ang_2_bohr
    
    f = open(filename, 'w')
    
    f.write('title\n')
    f.write('comment\n')
    
    dv_br = cell/cell_n
    
    f.write("%5d %12.6f %12.6f %12.6f\n"%(natoms, origin[0], origin[1], origin[2]))
    
    for i in range(3):
        f.write("%5d %12.6f %12.6f %12.6f\n"%(cell_n[i], dv_br[i][0], dv_br[i][1], dv_br[i][2]))
    
    for i in range(natoms):
        at_x = atoms[i][1] * ang_2_bohr
        at_y = atoms[i][2] * ang_2_bohr
        at_z = atoms[i][3] * ang_2_bohr
        f.write("%5d %12.6f %12.6f %12.6f %12.6f\n"%(atoms[i][4], 0.0, at_x, at_y, at_z))
    
    data.tofile(f, sep='\n', format='%12.6e')
    
    f.close()

In [52]:
positions = np.genfromtxt(file_xyz, dtype=None, skip_header=2)

write_cube_file(filename, positions, cell, cell_n, data)

2140520.5068390658

In [152]:
cell = np.array([[4.0, 0.0, 0.0], [0.0, 4.0, 0.0], [0.0, 0.0, 7.345]])
cell *= ang_2_bohr

step = 0.8
step *= ang_2_bohr

cell_n = np.array((int(cell[0, 0]/step), int(cell[1, 1]/step), int(cell[2, 2]/step)))

dv = cell/cell_n

x_at = cell[0,0]/2
y_at = cell[1,1]/2
z_at = cell[2,2]/2

data = np.zeros(cell_n) 

l = 1
m = 1

for xi in range(cell_n[0]):
    for yi in range(cell_n[1]):
        for zi in range(cell_n[2]):
            (x, y, z) = xi*dv[0] + yi*dv[1] + zi*dv[2]
            
            r_vec = np.array([x-x_at, y-y_at, z-z_at])
            r_vec_2 = np.dot(r_vec, r_vec)
            
            data[xi, yi, zi] = spherical_harmonic_real(l, m, r_vec)
            

In [156]:
vol_elem = np.linalg.det(cell/cell_n)
overlap(data, data, vol_elem)

2051.1720904340191

In [162]:
cell/cell_n

array([[ 1.5117809 ,  0.        ,  0.        ],
       [ 0.        ,  1.5117809 ,  0.        ],
       [ 0.        ,  0.        ,  1.54222649]])

In [157]:
data

array([[[-0.21081194, -0.24309668, -0.28020127, -0.31707081, -0.34195382,
         -0.34195382, -0.31707081, -0.28020127, -0.24309668],
        [-0.22461661, -0.26499207, -0.31534667, -0.3709798 , -0.4127047 ,
         -0.4127047 , -0.3709798 , -0.31534667, -0.26499207],
        [-0.23262002, -0.2784189 , -0.33872925, -0.4108013 , -0.46980417,
         -0.46980417, -0.4108013 , -0.33872925, -0.2784189 ],
        [-0.23262002, -0.2784189 , -0.33872925, -0.4108013 , -0.46980417,
         -0.46980417, -0.4108013 , -0.33872925, -0.2784189 ],
        [-0.22461661, -0.26499207, -0.31534667, -0.3709798 , -0.4127047 ,
         -0.4127047 , -0.3709798 , -0.31534667, -0.26499207]],

       [[-0.13476996, -0.15899524, -0.189208  , -0.22258788, -0.24762282,
         -0.24762282, -0.22258788, -0.189208  , -0.15899524],
        [-0.1449269 , -0.17647082, -0.22093591, -0.28020127, -0.33591986,
         -0.33591986, -0.28020127, -0.22093591, -0.17647082],
        [-0.15094954, -0.18768745, -0.24424331

3.5247300926800409